# Structurer un workflow pour le déployer en production

<img src="../images/workflow.png" alt="image workflow">

In [ ]:
import pandas as pd 
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
data = sns.load_dataset('tips')

In [ ]:
data.info()

In [ ]:
data.head(2)

In [ ]:
y = data['tip']

In [ ]:
X = data.drop(columns=['tip'])

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

# Iteration 1 - categoric features (get_dummies)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
#lr.fit(X_train, y_train)

In [ ]:
X_train.sample(5)

In [ ]:
X_train_dummies = pd.get_dummies(X_train)

In [ ]:
X_train_dummies.sample(4)

In [ ]:
lr_train = lr.fit(X_train_dummies, y_train)

In [ ]:
X_test_dummies = pd.get_dummies(X_test)

In [ ]:
lr_train.score(X_test_dummies, y_test)

# Iteration 2 - categoric + numeric features

In [ ]:
X_train_2,X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.3)

In [ ]:
sns.histplot(data=X_train_2[["total_bill"]], kde=True);

In [ ]:
sns.histplot(data=X_train_2[["size"]], kde=True)

## Scaler

### Numerical features

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train_2[["size","total_bill"]])
X_train_2_scaled = scaler.transform(X_train_2[["size","total_bill"]])

In [ ]:
X_train_2_scaled_df = pd.DataFrame(X_train_2_scaled).rename(columns={0: "size_scaled", 1: "total_bill_scaled"})

In [ ]:
X_train_2.head(20)

In [ ]:
X_train_2_scaled_df

### Categorical Features

In [ ]:
X_train_2[["sex","smoker","day","time"]].nunique()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe = OneHotEncoder()

In [ ]:
ohe.fit_transform(X_train_2[["sex","smoker","day","time"]])

In [ ]:
X_train_2_ohe = ohe.fit_transform(X_train_2[["sex","smoker","day","time"]])

In [ ]:
X_train_2_ohe_df = pd.DataFrame.sparse.from_spmatrix(X_train_2_ohe)

In [145]:
X_train_2_ohe_df.head(2)

,0,1,2,3,4,5,6,7,8,9
0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [ ]:
X_train_2_prep_df = pd.concat([X_train_2_ohe_df, X_train_2_scaled_df], axis=1)

In [ ]:
X_train_2_prep_df

In [ ]:
lr_2 = LinearRegression()

In [ ]:
lr_2.fit(X_train_2_prep_df,y_train_2)

In [147]:
#===> Il faut scalé le X_test_2_prep_df. Avec la même methodologie que le X_train_2, sans faire de dataLeakeage. 
# ==> Creéer les methodes appropriées pour réutiliser la logique.
X_test_2_scaled = scaler.transform(X_test_2[["size","total_bill"]])
X_test_2_scaled_df = pd.DataFrame(X_test_2_scaled).rename(columns={0: "size_scaled", 1: "total_bill_scaled"})
X_test_2_ohe = ohe.fit(X_train_2[["sex","smoker","day","time"]]).transform(X_test_2[["sex","smoker","day","time"]])
X_test_2_ohe_df = pd.DataFrame.sparse.from_spmatrix(X_test_2_ohe)
X_test_2_ohe_df.head(2)
X_test_2_prep_df = pd.concat([X_test_2_ohe_df, X_test_2_scaled_df], axis=1)
X_test_2_prep_df

,0,1,2,3,4,5,6,7,8,9,size_scaled,total_bill_scaled
0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.939394
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.2,0.053949
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.2,0.264692
3,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.2,0.230946
4,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.2,0.349862
...,...,...,...,...,...,...,...,...,...,...,...,...
69,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.6,0.303030
70,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.6,0.943067
71,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.2,-0.034435
72,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.2,0.055785


In [148]:
lr_2.score(X_test_2_prep_df, y_test_2)

/root/.pyenv/versions/simplon/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/root/.pyenv/versions/simplon/lib/python3.8/site-packages/sklearn/utils/validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


0.4517390901359032

In [ ]:
# Afficher un nuage de point qui révéle les informations suivantes : y_true vs y_predict en fonction de total_bill


In [ ]:
# Re-testester avec un StandardScaler => https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html



In [ ]:
# Afficher l'hétéroscédasticité ou l'homoscédasticité des erreurs.

In [ ]:
# Afficher la distribution des erreurs. (Normalité des erreus).

In [ ]:
# Créer un package à l'aide https://pypi.org/project/setuptools/